<a href="https://colab.research.google.com/github/isao1974/ai_calc/blob/master/ai_calc_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備
csvファイルからデータを読み込み学習させる。

csvファイルはgdrive上に用意するためgdriveをマウントする。
colaboratory 上でのCSVファイルの読み込みは下記を参照
https://qiita.com/uni-3/items/201aaa2708260cc790b8

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


コマンド
!ls など、!を頭につけると、コマンドが実行できる。

In [0]:
!conda list


/bin/bash: conda: command not found


# 学習用データ作成

In [0]:
import random
import csv
import pdb
import os
import sys
#pdb.set_trace()

home_dir = "drive/My Drive/Colab Notebooks/data/"
if os.path.isdir(home_dir) == False:
  print("file not found:" + home_dir)
  sys.exit()
    
# 加算データ
data_file = home_dir + "addData.csv"
with open(data_file, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['ope', '1st', '2nd', 'ans'])

  num = 0
  while num < 1000:
    val0 = 1
    val1 = random.randint(-100, 100)
    val2 = random.randint(-100, 100)
    val3 = val1 + val2
    writer.writerow([val0, val1, val2, val3])
    num += 1

#減算データ   
data_file = home_dir + "subData.csv"
with open(data_file, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['ope', '1st', '2nd', 'ans'])
    
  num = 0
  while num < 1000:
    val0 = 2
    val1 = random.randint(-100, 100)
    val2 = random.randint(-100, 100)
    val3 = val1 - val2
    writer.writerow([val0, val1, val2, val3])
    num += 1

#乗算データ    
data_file = home_dir + "mulData.csv"
with open(data_file, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['ope', '1st', '2nd', 'ans'])
    
  num = 0
  while num < 1000:
    val0 = 3
    val1 = random.randint(-100, 100)
    val2 = random.randint(-100, 100)
    val3 = val1 * val2
    writer.writerow([val0, val1, val2, val3])
    num += 1


#除算データ
data_file = home_dir + "devData.csv"
with open(data_file, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['ope', '1st', '2nd', 'ans'])
    
  num = 0
  while num < 1000:
    val0 = 4
    val1 = random.randint(-100, 100)
    val2 = random.randint(-100, 100)
    if val2 == 0:
      continue
    else:
      val3 = round(val1 / val2, 1)
    writer.writerow([val0, val1, val2, val3])
    num += 1


# 機械学習

学習済みモデルを使用する。
*   ph1: 加算する学習モデルを作成
*   ph2:四則演算する学習モデルを作成



https://qiita.com/Sa2Knight/items/221be46f2702ae453ba9
を参考に足し算を学習する学習モデルを作成する
モデルは保存する。

加算学習

In [0]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

formulas = DataFrame([
    [0, 0],
    [0, 1],
    [0, 2],
    [1, 0],
    [1, 1],
    [1, 2],
    [2, 0],
    [2, 1],
    [2, 2]
])
answers = DataFrame([0, 1, 2, 1, 2, 3, 2, 3, 4])

model = linear_model.LinearRegression()
model.fit(formulas, answers)

with open('model.pickle', mode='wb') as fp:
  pickle.dump(model, fp)
  

while True:
     # 標準入力から計算式を取得
     print('> ', end='')
     inStr = input()
    
     if inStr == 'q':
      break
        
     x, y = list(map(lambda x: int(x), inStr.split(' ')))

     # 学習モデルを用いて回答を取得し、標準出力
     predected_answer = model.predict([[x, y]])
     print("{0} + {1} = {2}".format(x, y, int(predected_answer[0][0])))

保存したモデルを使う

DataFrame の１番目を演算子として学習モデルを作る。
linear_model.LinearRegression だと期待する結果が得られなかったので、
様々な識別器を試してみる。

識別器一覧
https://qiita.com/yhyhyhjp/items/3ca101f1cb5dee310c06

結果は失敗。
原因推測
* 学習データが少ない？
* モデルのハイパーパラメータが調整できてない？
* 機械学習では無理？ディープラーニングだといける？

In [0]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

with open('model.pickle', mode='rb') as fp:
  model = pickle.load(fp)
  
  # 加算
  x, y = 10, 20
  predected_answer = model.predict([[x, y]])
  print("{0} + {1} = {3}".format(x, y, z, int(predected_answer[0])))

csv ファイルからデータ読み込み

In [0]:
import pandas as pd
from pandas import DataFrame
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
import sys
import os
import pickle

h = .02  # step size in the mesh

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]


# test file existence
testDataFile = "drive/My Drive/Colab Notebooks/data.csv"
if os.path.isfile(testDataFile) == False:
  print("file not found")
  sys.exit()
  

# read file and split data
data = pd.read_csv(testDataFile, header=0)
X = data.loc[:, ['ope', '1st', '2nd']]
Y = data.loc[:, ['ans']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)


#learning
for model in classifiers:
  print(model.__class__.__name__)  
  filename = "drive/My Drive/Colab Notebooks/"
  filename += model.__class__.__name__
  filename += ".pickle"
  print(filename)
  model.fit(X_train, Y_train)
  print(model.score(X_test, Y_test))
  print("-------")
  
  with open(filename, mode='wb') as fp:
    pickle.dump(model, fp)

学習モデルを使って推論
90%のスコアのSVCで推論させたが、数値が小さいときはいいけど、2桁になるともうダメ。

In [0]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

filename = "drive/My Drive/Colab Notebooks/"
filename += "SVC"
filename += ".pickle"


with open(filename, mode='rb') as fp:
  model = pickle.load(fp)
  
  # 加算
  ope, x, y = 0, 100, 200
  predected_answer = model.predict([[ope, x, y]])
  print("{0} ,{1}, {2} = {3}".format(ope, x, y, int(predected_answer[0])))

# deep learning で学習
* ph1: 加算する学習モデルを作成
* ph2:四則演算する学習モデルを作成

注意点
* 出力の活性化関数は linearを使う
* 損失関数は 二乗誤差を使う。交差エントロピーは確率で使うのでダメ。

四則演算を別々に学習
https://qiita.com/aidy91614/items/e90ee45fb82c1724d654


In [2]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import os
import pickle

def leaning(testDataFile, orgModel, epoch, modelFile):
  # test file existence
  if os.path.isfile(testDataFile) == False:
    print("file not found:" + testDataFile)
    sys.exit()

  config = orgModel.get_config()
  model = Sequential.from_config(config)
  # read file and split data
  data = pd.read_csv(testDataFile, header=0)
  X = data.loc[:, ['ope', '1st', '2nd']]
  Y = data.loc[:, ['ans']]
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

  model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['accuracy'])
  model.fit(X_train, Y_train, epochs=epoch, batch_size=64, verbose=0)
  score = model.evaluate(X_test, Y_test)
  print("loss=%f, acc=%f" % (score[0], score[1]))
  
  # save model
  with open(modelFile, mode='wb') as fp:
    pickle.dump(model, fp)

    
home_dir =  "drive/My Drive/Colab Notebooks/"

# construct model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=3))
model.add(Dense(1000, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))
plot_model(model, to_file=home_dir + "model/DL_Model.png", show_shapes=True)
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['accuracy'])

leaning(home_dir + "data/devData.csv", model, 2000, home_dir + "model/devModel.pickle")
leaning(home_dir + "data/mulData.csv", model, 2000, home_dir + "model/mulModel.pickle")
leaning(home_dir + "data/addData.csv", model, 2000, home_dir + "model/addModel.pickle")
leaning(home_dir + "data/subData.csv", model, 2000, home_dir + "model/subModel.pickle")

Using TensorFlow backend.
W0726 04:57:02.451878 139809719236480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 04:57:02.466619 139809719236480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 04:57:02.469204 139809719236480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 04:57:02.584430 139809719236480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 04:57:02.796923 13980971

200/200 [==============================] - 0s 275us/step
loss=31.024055, acc=0.115000
200/200 [==============================] - 0s 308us/step
loss=2865.266172, acc=0.000000
200/200 [==============================] - 0s 448us/step
loss=0.031020, acc=1.000000
200/200 [==============================] - 0s 534us/step
loss=0.005238, acc=1.000000


deep learning の学習モデルを使って推論する

python での四捨五入
https://note.nkmk.me/python-round-decimal-quantize/

フォーマット
https://www.headboost.jp/python-how-to-handle-digits/

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import plot_model
from pandas import DataFrame
import pickle
import numpy as np
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN

#load models
home_dir = "drive/My Drive/Colab Notebooks/model/"

addFp = open(home_dir + "addModel.pickle", mode='rb')
addModel = pickle.load(addFp)

subFp = open(home_dir + "subModel.pickle", mode='rb')
subModel = pickle.load(subFp)

mulFp = open(home_dir + "mulModel.pickle", mode='rb')
mulModel = pickle.load(mulFp)

devFp = open(home_dir + "devModel.pickle", mode='rb')
devModel = pickle.load(devFp)

while True:
  # 標準入力から計算式を取得
  print('> ', end='')
  inStr = input()

  if inStr == 'q':
    print("exit")
    break

  ope, x, y = list(map(lambda x: int(x), inStr.split(' ')))

  # 学習モデルを用いて回答を取得し、標準出力
  if ope == 1:
    predected_answer = addModel.predict(np.array([[ope, x, y]]))
    print("{0} + {1} = {2}".format(x, y, int(predected_answer[0])))
  elif ope == 2:
    predected_answer = subModel.predict(np.array([[ope, x, y]]))
    print("{0} - {1} = {2}".format(x, y, int(predected_answer[0])))
  elif ope == 3:
    predected_answer = mulModel.predict(np.array([[ope, x, y]]))
    print("{0} * {1} = {2}".format(x, y, int(predected_answer[0])))
  elif ope == 4:
    predected_answer = devModel.predict(np.array([[ope, x, y]]))
    print("{0} / {1} = {2}".format(x, y, int(predected_answer[0])))
  else:
    print("unknown operater:{0}".format(ope))

      
addFp.close()
subFp.close()
mulFp.close()
devFp.close()

Using TensorFlow backend.
W0726 05:30:17.670423 139927840696192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 05:30:17.688233 139927840696192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 05:30:17.750643 139927840696192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0726 05:30:17.751746 139927840696192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0726 05:30:17.75

> 1 2 3
2 + 3 = 5
> 1 5 5
5 + 5 = 10
> 1 10 10
10 + 10 = 20
> 1 200 10
200 + 10 = 210
> 2 300 10
300 - 10 = 290
> 2 400 20
400 - 20 = 380
> 3 10 10
10 * 10 = 87
> 3 10 200
10 * 200 = 1116
> 4 6 3
6 / 3 = 1
> 